In [1]:
!pip install -q git+https://github.com/sunsmarterjie/yolov12.git
!pip install -q flash-attn supervision

!pip uninstall -y tensorboard tensorflow

print("Done YOLOv12")

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 107.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 100.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 90.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 9.1 MB

In [2]:
# 4. QUAN TRỌNG: Bây giờ mới hạ cấp numpy xuống 1.26.4
!pip install -q numpy==1.26.4

print("Done numpy 1.26.4.")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 106.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dopamine-rl 4.1.2 requires tensorflow>=2.2.0, which is not installed.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
tensorflow-decision-forests 1.11.0 requires tensorflow==2.18.0, which is not installed.
datasets 4.4.1 requires pyarrow>=21.0.0, but you have pyarrow 19.0.1 which is incompatible.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
google-colab 1.0.0 requires notebook==6.5.7, but you have notebook 6.5.4 which is incompatible.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.
google-colab 1.0.0 requires requests==2.32

In [3]:
import shutil
import os

# Đường dẫn gốc (Read-only)
src_dir = '/kaggle/input/yolo-trash-dataset/final_yolo_trash-dataset'

dst_dir = '/tmp/yolo-trash-dataset' 

print(f"Copy from {src_dir} to {dst_dir} ")
if not os.path.exists(dst_dir):
    shutil.copytree(src_dir, dst_dir)
print("Copied")

Copy from /kaggle/input/yolo-trash-dataset/final_yolo_trash-dataset to /tmp/yolo-trash-dataset 
Copied


In [4]:
import textwrap

# Trỏ về thư mục mới trong /tmp
dataset_path = '/tmp/yolo-trash-dataset'

yaml_content = f"""
path: {dataset_path}
train: images/train
val: images/val
test: images/test

nc: 1
names:
  0: trash
"""

with open('/kaggle/working/yolo_trash_dataset.yaml', 'w') as f:
    f.write(textwrap.dedent(yaml_content))
    
print("Done YAML")

Done YAML


In [5]:
from ultralytics import YOLO
import torch

torch.cuda.empty_cache()

model = YOLO('yolov12n.pt') 

print("Bắt đầu huấn luyện:\n")

results = model.train(
    data='/kaggle/working/yolo_trash_dataset.yaml',
    
    epochs=400,
    patience=20,
    
    batch=56,
    imgsz=640,
    workers=4,
    
    cache='disk',          
    amp=True,
    optimizer='auto',
    
    close_mosaic=20,
    flipud=0.0,
    fliplr=0.5,
    
    project='/kaggle/working/yolov12_trash_training_runs',
    name='yolov12_trash_detection_model',
    exist_ok=True,
    plots=True,
    save=True
)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/yolov12/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
FlashAttention is not available on this device. Using scaled_dot_product_attention instead.


100%|██████████| 5.26M/5.26M [00:00<00:00, 70.0MB/s]


Bắt đầu huấn luyện:

New https://pypi.org/project/ultralytics/8.3.237 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.63 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov12n.pt, data=/kaggle/working/yolo_trash_dataset.yaml, epochs=400, time=None, patience=20, batch=56, imgsz=640, save=True, save_period=-1, cache=disk, device=None, workers=4, project=/kaggle/working/yolov12_trash_training_runs, name=yolov12_trash_detection_model, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=20, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=

100%|██████████| 755k/755k [00:00<00:00, 16.4MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      2368  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2, 1, 2]          
  2                  -1  1      6640  ultralytics.nn.modules.block.C3k2            [32, 64, 1, False, 0.25]      
  3                  -1  1      9344  ultralytics.nn.modules.conv.Conv             [64, 64, 3, 2, 1, 4]          
  4                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  5                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  6                  -1  2    174720  ultralytics.nn.modules.block.A2C2f           [128, 128, 2, True, 4]        
  7                  -1  1    295424  ultralytics

...


71 epochs completed in 1.061 hours.
Optimizer stripped from /kaggle/working/yolov12_trash_training_runs/yolov12_trash_detection_model/weights/last.pt, 5.4MB
Optimizer stripped from /kaggle/working/yolov12_trash_training_runs/yolov12_trash_detection_model/weights/best.pt, 5.4MB

Validating /kaggle/working/yolov12_trash_training_runs/yolov12_trash_detection_model/weights/best.pt...
Ultralytics 8.3.63 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLOv12n summary (fused): 376 layers, 2,508,539 parameters, 0 gradients, 5.8 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:06<00:00,  1.13s/it]


                   all        666        980      0.983      0.997      0.992       0.82


/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


Speed: 0.1ms preprocess, 5.4ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to /kaggle/working/yolov12_trash_training_runs/yolov12_trash_detection_model
